In [ ]:
import os
import pandas as pd
from datasets import load_dataset
import ast

## Load the data from Huggingface

In [ ]:
def convert_labels(example):
    example["Labels"] = [i for i in ast.literal_eval(example["Labels"])]
    return example

valueeval23 = load_dataset("webis/Touche23-ValueEval")
training_dataset = valueeval23["training"].map(convert_labels)
validation_dataset = valueeval23["validation"].map(convert_labels)
# training_dataset['Labels']
valueeval23

## Load data from local files

Data PATH

In [ ]:
data_dir = os.path.join('task_1','dataset')
arguments_training_filepath = os.path.join(data_dir, 'arguments-training.tsv')
arguments_validation_filepath = os.path.join(data_dir, 'arguments-validation.tsv')
arguments_validation_filepath_zhihu = os.path.join(data_dir, 'arguments-validation-zhihu.tsv')
arguments_test_filepath = os.path.join(data_dir, 'arguments-test.tsv')

labels_training_filepath = os.path.join(data_dir, 'labels-training.tsv')
labels_validation_filepath = os.path.join(data_dir, 'labels-validation.tsv')
labels_validation_filepath_zhihu = os.path.join(data_dir, 'labels-validation-zhihu.tsv')
labels_test_filepath = os.path.join(data_dir, 'labels-test.tsv')

Load the data

In [ ]:
arguments_training = pd.read_csv(arguments_training_filepath, encoding='utf-8', sep='\t', header=0)
arguments_validation = pd.read_csv(arguments_validation_filepath, encoding='utf-8', sep='\t', header=0)
arguments_validation_zhihu = pd.read_csv(arguments_validation_filepath_zhihu, encoding='utf-8', sep='\t', header=0)
arguments_test = pd.read_csv(arguments_test_filepath, encoding='utf-8', sep='\t', header=0)


labels_training = pd.read_csv(labels_training_filepath, encoding='utf-8', sep='\t', header=0)
labels_validation = pd.read_csv(labels_validation_filepath, encoding='utf-8', sep='\t', header=0)
labels_validation_zhihu = pd.read_csv(labels_validation_filepath_zhihu, encoding='utf-8', sep='\t', header=0)
labels_test = pd.read_csv(labels_test_filepath, encoding='utf-8', sep='\t', header=0)


print(arguments_training.iloc[0], '\n', labels_training.iloc[0])

## Some helper functions

In [ ]:
def convert_binary_labels_to_string(df):
    label_names = df.columns[1:]
    string_labels = []

    for index, row in df.iterrows():
        binary_values = row.values[1:]
        string_labels.append([label_names[i] for i, value in enumerate(binary_values) if value == 1])

    df['String Labels'] = string_labels
    return df

def add_prompt_to_df(df):
    prompt_format = "Premise: {}\n Stance: {}\n Conclusion {}\n. Which value category does it support?\n"
    preprocessed_arguments = []

    for _, row in df.iterrows():
        premise = row['Premise']
        stance = row['Stance']
        conclusion = row['Conclusion']
        prompt = prompt_format.format(premise, stance, conclusion)
        preprocessed_arguments.append(prompt)
        
    df['Prompt'] = preprocessed_arguments
    return df

def combine_columns(df_arguments, df_labels):
    """Combines the two `DataFrames` on column `Argument ID`"""
    # Combine the two dataframes use for the labels only the column 'String Labels'
    df_labels = df_labels[['Argument ID', 'String Labels']]
    df_labels.columns = ['Argument ID', 'Labels']

    return pd.merge(df_arguments, df_labels, on='Argument ID')

def labels_to_multi_choice(labels):
    """Converts the labels to a multi choice format"""
    multi_choice_format = "{}: {}"
    multi_choice_options = []

    for index, label in enumerate(labels):
        multi_choice_option = multi_choice_format.format(chr(65 + index), label)
        multi_choice_options.append(multi_choice_option)

    return multi_choice_options

# Add Prompt to the datafram and Convert the label to string

In [ ]:
# get the unique values for labels
# print(labels_training.columns.unique())

# use the unique calums as multi choice options for the prompt eg A: 1, B: 2, C: 3, D: 4

labels = labels_training.columns.unique()
print(labels)
# remove Argument ID and String Labels from the list
labels = [label for label in labels if label != 'Argument ID' and label != 'String Labels']

multi_choice_options = labels_to_multi_choice(labels)
print(labels,multi_choice_options)

# can you generate a promt for argument consisting of premise, stance and conclusion where the model needs to predict a category value consisting out of 'Self-direction: thought', 'Self-direction: action',
#        'Stimulation', 'Hedonism', 'Achievement', 'Power: dominance',
#        'Power: resources', 'Face', 'Security: personal', 'Security: societal',
#        'Tradition', 'Conformity: rules', 'Conformity: interpersonal',
#        'Humility', 'Benevolence: caring', 'Benevolence: dependability',
#        'Universalism: concern', 'Universalism: nature',
#        'Universalism: tolerance', 'Universalism: objectivity'

In [ ]:
arguments_training = add_prompt_to_df(arguments_training)
converted_labels = convert_binary_labels_to_string(labels_training)
training_arg_labels = combine_columns(arguments_training, converted_labels)

arguments_training['Prompt'].iloc[0]


Combine the arguments and string labels into df

In [ ]:
training_arg_labels.head()

In [ ]:
argument = arguments_training.iloc[1]
print(argument)
sample = f"The premise: {argument['Premise']} is {argument['Stance']} conclusion: {argument['Conclusion']}. Which value category does it support? \n (A) societal (B) rules (C) interpersonal (D) none of the above"
sample

In [ ]:
argument = processed_arguments[1]
argument = f"{argument} (A) societal (B) rules (C) interpersonal (D) none of the above"
argument

In [ ]:
# from typing import Optional, List
# from transformers import pipeline

# from transformers import pipeline

# generator = pipeline("text2text-generation", model="allenai/unifiedqa-t5-base")

In [ ]:
pwdproject/LTP_Project_Group_6/

In [ ]:
from datasets import load_dataset, Dataset
dataset_path = 'task_1/dataset/processed/dataset_dict/'


